<a href="https://colab.research.google.com/github/traopia/KGNarrative/blob/master/esperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mining News' Entities


In [1]:
!pip install transformers 
!pip install datasets
!pip install pynvml
!pip install evaluate 
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 3.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-pyth

In [2]:
import transformers
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import os
import nltk
import torch
import evaluate
import sys
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
from sklearn.model_selection import train_test_split

In [ ]:
#! git clone https://github.com/traopia/KGNarrative.git

In [3]:
# define utils functions to facilitate gpu 

def check_gpu_availability():
    # Check if CUDA is available
    print(f"Cuda is available: {torch.cuda.is_available()}")

def getting_device(gpu_prefence=True) -> torch.device:
    """
    This function gets the torch device to be used for computations, 
    based on the GPU preference specified by the user.
    """
    
    # If GPU is preferred and available, set device to CUDA
    if gpu_prefence and torch.cuda.is_available():
        device = torch.device('cuda')
    # If GPU is not preferred or not available, set device to CPU
    else: 
        device = torch.device("cpu")
    
    # Print the selected device
    print(f"Selected device: {device}")
    
    # Return the device
    return device

# Define a function to print GPU memory utilization
def print_gpu_utilization():
    # Initialize the PyNVML library
    nvmlInit()
    # Get a handle to the first GPU in the system
    handle = nvmlDeviceGetHandleByIndex(0)
    # Get information about the memory usage on the GPU
    info = nvmlDeviceGetMemoryInfo(handle)
    # Print the GPU memory usage in MB
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

# Define a function to print training summary information
def print_summary(result):
    # Print the total training time in seconds
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    # Print the number of training samples processed per second
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    # Print the GPU memory utilization
    print_gpu_utilization()

In [4]:
# CHECK IF GPU IS UP
check_gpu_availability()



Cuda is available: True


In [5]:
# SAVE THE DEVICE WE ARE WORKING WITH
device = getting_device(gpu_prefence=True)

Selected device: cuda


#### Importing documents


In [6]:
path2file = "/content/DWIE_train_kg_topic_summary_news.csv.csv"
df = pd.read_csv(path2file)


In [7]:
df.head(10)

,Unnamed: 0,story,Instances Knowledge Graph,Types Knowledge Graph,Subclass Knowledge Graph,predicted_label1,core description,mined kg with ner
0,0,Multi-lingual in the World Wide Web Profession...,Personal Translator - product_of - Linguatec |...,Personal Translator - type - entity | Personal...,misc - subclass_of - entity | product - subcla...,tech,– If you’ve ever typed an email to a colleague...,{Windows - type - ORG | three - type - CARDINA...
1,1,German Know-How For China's Energy Sector Ener...,Shangdong Linuo Paradigma - based_in0 - German...,China - type - entity | China - type - gpe | C...,gpe - subclass_of - location | gpe0 - subclass...,business,– Germany and China have opened the world's la...,"{more than 6,000 - type - CARDINAL | Chinese -..."
2,2,Trash crisis forces Lebanon's environmental aw...,Kassem Kazak - citizen_of - Lebanon | Kassem K...,Kareem Chehayeb - type - entity | Kareem Cheha...,journalist - subclass_of - per | manager - sub...,tech,– When Kassem Kazak set out to start a recycli...,{the summer of 2015 - type - DATE | July 2015 ...
3,3,"Iran calls for end to Saudi air campaign, as U...",Houthi - based_in0 - Yemen | Houthi - based_in...,Medecins Sans Frontieres - type - entity | Med...,ngo - subclass_of - org | gpe - subclass_of - ...,business,– Iran's president has called for an end to ai...,{a week ago - type - DATE | Habibollah Sajjari...
4,4,Ai Weiwei Drifting Ai Weiwei: Uncomfortable cr...,DW (Deutsch+) - based_in0 - Germany | Ai Weiwe...,DW (Deutsch+) - type - entity | DW (Deutsch+) ...,media - subclass_of - org | activist - subclas...,entertainment,– Chinese dissident artist and human rights ac...,{Mao - type - PERSON | Turkey - type - GPE | E...
5,5,"Journal Interview with Günter Nooke, The Chanc...",Günter Nooke - agent_of - Germany | Günter Noo...,Günter Nooke - type - entity | Günter Nooke - ...,gov_per - subclass_of - per | location - subcl...,politics,– German Chancellor Angela Merkel's office say...,{this week - type - DATE | German - type - NOR...
6,6,Turkey summons German envoy for second time in...,Martin Erdmann - agent_of - Germany | Martin E...,European Union - type - entity | European Unio...,igo - subclass_of - org | so - subclass_of - i...,politics,– Relations between Turkey and Germany have be...,{Peter Steudtner - type - PERSON | 17th - type...
7,7,Medvedev seeks investment at Davos economic fo...,Dmitry Medvedev - agent_of - Russia | Dmitry M...,Dmitry Medvedev - type - entity | Dmitry Medve...,head_of_gov - subclass_of - politician | polit...,business,– The World Economic Forum in Davos has kicked...,"{Moscow - type - GPE | roughly 2,500 - type - ..."
8,8,Ex-CIA fugitive Robert Seldon Lady detained in...,Osama Moustafa Hassan Nasr - citizen_of - Egyp...,Osama Moustafa Hassan Nasr - type - clergy | O...,offender - subclass_of - per | agency - subcla...,business,– A former CIA station chief convicted in abse...,{23 - type - CARDINAL | Robert Seldon - type -...
9,9,"Nearly 10,000 migrants rescued on Mediterranea...",Rome - in0 - Italy | Rome - in0-x - Italian | ...,Strait of Sicily - type - entity | Strait of S...,location - subclass_of - entity | waterbody - ...,business,– The Italian coast guard and the EU's border ...,{the same month last year - type - DATE | Medi...


#### Trying Out NER pipeline


In [8]:
!pip install flair 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 42.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 KB 70.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━

In [13]:
from flair.models import SequenceTagger
from flair.data import Sentence

flairModelNer = 'ner-ontonotes-fast' # other option =>  'ner-ontonotes', 'ner-ontonotes-large'
flairModelPos = "pos-fast"

# Load the NER model
taggerNer = SequenceTagger.load(flairModelNer)
taggerPos = SequenceTagger.load(flairModelPos)

2023-03-08 15:59:41,070 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


2023-03-08 15:59:42,588 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [19]:
# WITHOUT POS

# create a sentence item
text = Sentence(df["story"][1])

# predict NER tags on the sentence
taggerNer.predict(text)

# print again the sentence item with predicted tags
for entity in text.get_spans('ner'):
    print(entity)

Span[0:1]: "German" → NORP (0.8688)
Span[3:4]: "China" → GPE (0.9984)
Span[5:8]: "Energy Sector Energy" → ORG (0.8701)
Span[14:15]: "China" → GPE (0.9999)
Span[16:17]: "Beijing" → GPE (1.0)
Span[23:24]: "Friday" → DATE (0.9987)
Span[25:26]: "Germany" → GPE (0.9999)
Span[34:35]: "German-Chinese" → NORP (0.9017)
Span[45:46]: "Jinan" → GPE (0.9998)
Span[64:65]: "Chinese" → NORP (0.9991)
Span[70:71]: "Sino-German" → NORP (0.6987)
Span[98:100]: "this year" → DATE (0.8376)
Span[101:104]: "30 million kilowatts" → QUANTITY (0.8853)
Span[106:107]: "China" → GPE (0.9997)
Span[127:128]: "German" → NORP (0.9975)
Span[130:132]: "Joschka Fischer" → PERSON (0.9971)
Span[133:134]: "German" → NORP (0.9993)
Span[147:148]: "China" → GPE (0.9999)
Span[181:182]: "German" → NORP (0.9995)
Span[187:188]: "Paradigma" → ORG (0.9729)
Span[189:190]: "Karlsbad" → GPE (0.9177)
Span[192:193]: "Chinese" → NORP (0.9978)
Span[196:197]: "Linuo" → ORG (0.9251)
Span[198:199]: "Jinan" → GPE (0.9902)
Span[200:201]: "Paradig

In [31]:
# WITH POS

# create a sentence item
text = Sentence(df["story"][1])

# predict NER and POS tags on the sentence
taggerNer.predict(text)
taggerPos.predict(text)

# iterate through the entity spans
for entity in text.get_spans("ner"):
  # get the tokens within the entity span
  tokens = entity.tokens
  # check if the entity has a NER label
  if entity.tag:
    # iterate through the tokens and get their POS labels
    pos_labels = [token.get_labels()[0].value for token in tokens]
    # print the entity and its NER and POS labels
    print( entity.text, entity.tag, pos_labels[0])


German NORP JJ
China GPE NNP
Energy Sector Energy ORG NNP
China GPE NNP
Beijing GPE NNP
Friday DATE NNP
Germany GPE NNP
German-Chinese NORP JJ
Jinan GPE NNP
Chinese NORP JJ
Sino-German NORP JJ
this year DATE DT
30 million kilowatts QUANTITY CD
China GPE NNP
German NORP JJ
Joschka Fischer PERSON NNP
German NORP JJ
China GPE NNP
German NORP JJ
Paradigma ORG NNP
Karlsbad GPE NNP
Chinese NORP JJ
Linuo ORG NNP
Jinan GPE NNP
Paradigma ORG NNP
Alfred Ritter PERSON NNP
Ritter Sport ORG NNP
Ritter PERSON NNP
China GPE NNP
Asian NORP JJ
Germany GPE NNP
European NORP JJ
Germany GPE NNP
Shangdong GPE NNP
Linuo Paradigma ORG NNP
German NORP JJ
Fischer PERSON NNP
Asia LOC NNP
German NORP JJ
China GPE NNP
China GPE NNP
the 1980s DATE DT
four CARDINAL CD
China GPE NNP
China GPE NNP
Beijing GPE NNP
night TIME NN
more than 6,000 CARDINAL JJR
Beijing GPE NNP
this summer DATE DT


In [ ]:
# WITHOUT POS

def extract_named_entities(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    entities = []
    for entity in sentence.get_spans('ner'):
        entity_text = entity.text
        entity_type = entity.labels[0].value
        entities.append(f"{entity_text} rdf:type {entity_type}")
    return ', '.join(entities)

extract_named_entities(df["story"][1])

'German rdf:type NORP, China rdf:type GPE, Energy Sector Energy rdf:type ORG, China rdf:type GPE, Beijing rdf:type GPE, Friday rdf:type DATE, Germany rdf:type GPE, German-Chinese rdf:type NORP, Jinan rdf:type GPE, Chinese rdf:type NORP, Sino-German rdf:type NORP, this year rdf:type DATE, 30 million kilowatts rdf:type QUANTITY, China rdf:type GPE, German rdf:type NORP, Joschka Fischer rdf:type PERSON, German rdf:type NORP, China rdf:type GPE, German rdf:type NORP, Paradigma rdf:type ORG, Karlsbad rdf:type GPE, Chinese rdf:type NORP, Linuo rdf:type ORG, Jinan rdf:type GPE, Paradigma rdf:type ORG, Alfred Ritter rdf:type PERSON, Ritter Sport rdf:type ORG, Ritter rdf:type PERSON, China rdf:type GPE, Asian rdf:type NORP, Germany rdf:type GPE, European rdf:type NORP, Germany rdf:type GPE, Shangdong rdf:type GPE, Linuo Paradigma rdf:type ORG, German rdf:type NORP, Fischer rdf:type PERSON, Asia rdf:type LOC, German rdf:type NORP, China rdf:type GPE, China rdf:type GPE, the 1980s rdf:type DATE, 

In [35]:
# WITH POS 
def extract_named_entities(text):
    sentence = Sentence(text)
    taggerNer.predict(sentence)
    taggerPos.predict(sentence)
    entities = []
    for entity in sentence.get_spans('ner'):
        entity_text = entity.text
        entity_type = entity.labels[0].value
        tokens = entity.tokens
        pos_label = [token.get_labels()[0].value for token in tokens][0]

        # FILTERING OUT ADJECTIVES 
        if pos_label != "JJ" and pos_label != "JJR":
          entities.append(f"{entity_text} rdf:type {entity_type}_{pos_label}")
    return ', '.join(entities)

extract_named_entities(df["story"][1])

'China rdf:type GPE_NNP, Energy Sector Energy rdf:type ORG_NNP, China rdf:type GPE_NNP, Beijing rdf:type GPE_NNP, Friday rdf:type DATE_NNP, Germany rdf:type GPE_NNP, Jinan rdf:type GPE_NNP, this year rdf:type DATE_DT, 30 million kilowatts rdf:type QUANTITY_CD, China rdf:type GPE_NNP, Joschka Fischer rdf:type PERSON_NNP, China rdf:type GPE_NNP, Paradigma rdf:type ORG_NNP, Karlsbad rdf:type GPE_NNP, Linuo rdf:type ORG_NNP, Jinan rdf:type GPE_NNP, Paradigma rdf:type ORG_NNP, Alfred Ritter rdf:type PERSON_NNP, Ritter Sport rdf:type ORG_NNP, Ritter rdf:type PERSON_NNP, China rdf:type GPE_NNP, Germany rdf:type GPE_NNP, Germany rdf:type GPE_NNP, Shangdong rdf:type GPE_NNP, Linuo Paradigma rdf:type ORG_NNP, Fischer rdf:type PERSON_NNP, Asia rdf:type LOC_NNP, China rdf:type GPE_NNP, China rdf:type GPE_NNP, the 1980s rdf:type DATE_DT, four rdf:type CARDINAL_CD, China rdf:type GPE_NNP, China rdf:type GPE_NNP, Beijing rdf:type GPE_NNP, night rdf:type TIME_NN, Beijing rdf:type GPE_NNP, this summer 

In [ ]:
# Getting the right format and unique entities
def extract_named_unique_entities(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    entities = []
    for entity in sentence.get_spans('ner'):
        entities.append((entity.text,entity.labels[0].value))  
    entities = tuple(set(entities))
    output = '{'
    for entity in entities:
      output += entity[0] + ' - type - ' + entity[1] + ' | '
    output = output[:-3] + '}' 
    return output

extract_named_entities(df["story"][1])

'{more than 6,000 - type - CARDINAL | Chinese - type - NORP | Asian - type - NORP | 30 million kilowatts - type - QUANTITY | Jinan - type - GPE | Fischer - type - PERSON | Shangdong - type - GPE | Linuo Paradigma - type - ORG | Asia - type - LOC | European - type - NORP | night - type - TIME | Ritter - type - PERSON | German-Chinese - type - NORP | Alfred Ritter - type - PERSON | Paradigma - type - ORG | Sino-German - type - NORP | Ritter Sport - type - ORG | this year - type - DATE | China - type - GPE | this summer - type - DATE | four - type - CARDINAL | the 1980s - type - DATE | Friday - type - DATE | Germany - type - GPE | German - type - NORP | Karlsbad - type - GPE | Energy Sector Energy - type - ORG | Linuo - type - ORG | Joschka Fischer - type - PERSON | Beijing - type - GPE}'

In [45]:
# WITH POS FILTERING AND NER TYPE 

# Getting the right format and unique entities
def extract_named_unique_entities_with_filters(text):
    sentence = Sentence(text)
    taggerNer.predict(sentence)
    taggerPos.predict(sentence)
    entities = []
    for entity in sentence.get_spans('ner'):
        entity_text = entity.text
        entity_type = entity.labels[0].value
        tokens = entity.tokens
        pos_label = [token.get_labels()[0].value for token in tokens][0]

        # FILTERING OUT ADJECTIVES 
        if pos_label != "JJ" and pos_label != "JJR":
          if entity_type != "PERCENT" and entity_type != "QUANTITY" and entity_type != "CARDINAL":
            entities.append((entity.text,entity.labels[0].value))  
    entities = tuple(set(entities))
    output = '{'
    for entity in entities:
      output += entity[0] + ' - type - ' + entity[1] + ' | '
    output = output[:-3] + '}' 
    return output

extract_named_unique_entities_with_filters(df["story"][1])

'{Alfred Ritter - type - PERSON | Linuo - type - ORG | Energy Sector Energy - type - ORG | Paradigma - type - ORG | Linuo Paradigma - type - ORG | Beijing - type - GPE | Karlsbad - type - GPE | Shangdong - type - GPE | this summer - type - DATE | Ritter - type - PERSON | China - type - GPE | the 1980s - type - DATE | night - type - TIME | Friday - type - DATE | this year - type - DATE | Joschka Fischer - type - PERSON | Ritter Sport - type - ORG | Asia - type - LOC | Jinan - type - GPE | Germany - type - GPE | Fischer - type - PERSON}'

#### Extracting NER and adding it to the csv

In [43]:
# df.drop(['mined kg with ner'], axis = 1, inplace = True) 

In [46]:
df['mined_kg_entities'] = df['story'].apply(extract_named_unique_entities_with_filters)

In [47]:
print(print_gpu_utilization())

GPU memory occupied: 2165 MB.
None


In [48]:
df["mined_kg_entities"][100]

'{Red Crescent - type - ORG | Maysa al-Aswad - type - PERSON | Narjis - type - PERSON | Aleppo - type - GPE | Fuaa - type - GPE | Sunday - type - DATE | Syrian - type - NORP | AFP - type - ORG | Rashidin - type - GPE | the next two months - type - DATE | Hadi - type - PERSON | Reuters - type - ORG | Kafraya - type - GPE | Ahrar al-Sham - type - ORG | the last few years - type - DATE | the Syrian Observatory for Human Rights - type - ORG | a day - type - DATE | Observatory - type - ORG | Syria - type - GPE | Saturday - type - DATE}'

In [49]:
df.head()

,Unnamed: 0,story,Instances Knowledge Graph,Types Knowledge Graph,Subclass Knowledge Graph,predicted_label1,core description,mined_kg_entities
0,0,Multi-lingual in the World Wide Web Profession...,Personal Translator - product_of - Linguatec |...,Personal Translator - type - entity | Personal...,misc - subclass_of - entity | product - subcla...,tech,– If you’ve ever typed an email to a colleague...,{ten years ago - type - DATE | the American Sm...
1,1,German Know-How For China's Energy Sector Ener...,Shangdong Linuo Paradigma - based_in0 - German...,China - type - entity | China - type - gpe | C...,gpe - subclass_of - location | gpe0 - subclass...,business,– Germany and China have opened the world's la...,{Alfred Ritter - type - PERSON | Linuo - type ...
2,2,Trash crisis forces Lebanon's environmental aw...,Kassem Kazak - citizen_of - Lebanon | Kassem K...,Kareem Chehayeb - type - entity | Kareem Cheha...,journalist - subclass_of - per | manager - sub...,tech,– When Kassem Kazak set out to start a recycli...,{Lebanon - type - GPE | Sukleen - type - PERSO...
3,3,"Iran calls for end to Saudi air campaign, as U...",Houthi - based_in0 - Yemen | Houthi - based_in...,Medecins Sans Frontieres - type - entity | Med...,ngo - subclass_of - org | gpe - subclass_of - ...,business,– Iran's president has called for an end to ai...,{PBS - type - ORG | the Gulf of Aden - type - ...
4,4,Ai Weiwei Drifting Ai Weiwei: Uncomfortable cr...,DW (Deutsch+) - based_in0 - Germany | Ai Weiwe...,DW (Deutsch+) - type - entity | DW (Deutsch+) ...,media - subclass_of - org | activist - subclas...,entertainment,– Chinese dissident artist and human rights ac...,{Einstein - type - PERSON | Europe - type - LO...


In [52]:
# Save the updated dataframe
df.to_csv("DWIE_train_entities_topic_summary_news.csv", index=False)

#### 